In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import sys

In [2]:
import TensorflowUtils as utils
import read_BRATSParsingData as scene_parsing
import datetime
import BatchDatsetReader_3modalities as dataset
from six.moves import xrange

In [3]:
data_dirs = "/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG"
test_dirs = "/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG"

In [4]:
FLAGS = tf.flags.FLAGS
tf.flags.DEFINE_integer("batch_size", 1, "batch size for training")
tf.flags.DEFINE_string("logs_dir", "logs/", "path to logs directory")
tf.flags.DEFINE_string("data_dir", data_dirs, "path to dataset")
tf.flags.DEFINE_string("test_dir", test_dirs, "path to testset")
tf.flags.DEFINE_float("learning_rate", 1e-4, "Learning rate for Adam Optimizer")
tf.flags.DEFINE_string("model_dir", "./", "Path to vgg model mat")
tf.flags.DEFINE_bool('debug', False, "Debug mode: True/ False")
tf.flags.DEFINE_string('mode', "train", "Mode train/ test/ valid_visualize/ test_visualize")
MODEL_URL = 'http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat'

In [5]:
MAX_ITERATION = int(10000 + 1)
NUM_OF_CLASSESS = 5
IMAGE_SIZE = 224

In [6]:
def vgg_net(weights, image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    net = {}
    current = image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = utils.get_variable(np.transpose(kernels, (1, 0, 2, 3)), name=name + "_w")
            bias = utils.get_variable(bias.reshape(-1), name=name + "_b")
            current = utils.conv2d_basic(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current, name=name)
            if FLAGS.debug:
                utils.add_activation_summary(current)
        elif kind == 'pool':
            current = utils.avg_pool_2x2(current)
        net[name] = current

    return net

In [7]:
def inference(image, keep_prob):
    """
    Semantic segmentation network definition
    :param image: input image. Should have values in range 0-255
    :param keep_prob:
    :return:
    """
    print("setting up vgg initialized conv layers ...")
    model_data = utils.get_model_data(FLAGS.model_dir, MODEL_URL)

    mean = model_data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))

    weights = np.squeeze(model_data['layers'])

    processed_image = utils.process_image(image, mean_pixel)

    with tf.variable_scope("inference"):
        image_net = vgg_net(weights, processed_image)
        conv_final_layer = image_net["conv5_3"]

        pool5 = utils.max_pool_2x2(conv_final_layer)

        W6 = utils.weight_variable([7, 7, 512, 4096], name="W6")
        b6 = utils.bias_variable([4096], name="b6")
        conv6 = utils.conv2d_basic(pool5, W6, b6)
        relu6 = tf.nn.relu(conv6, name="relu6")
        if FLAGS.debug:
            utils.add_activation_summary(relu6)
        relu_dropout6 = tf.nn.dropout(relu6, keep_prob=keep_prob)

        W7 = utils.weight_variable([1, 1, 4096, 4096], name="W7")
        b7 = utils.bias_variable([4096], name="b7")
        conv7 = utils.conv2d_basic(relu_dropout6, W7, b7)
        relu7 = tf.nn.relu(conv7, name="relu7")
        if FLAGS.debug:
            utils.add_activation_summary(relu7)
        relu_dropout7 = tf.nn.dropout(relu7, keep_prob=keep_prob)

        W8 = utils.weight_variable([1, 1, 4096, NUM_OF_CLASSESS], name="W8")
        b8 = utils.bias_variable([NUM_OF_CLASSESS], name="b8")
        conv8 = utils.conv2d_basic(relu_dropout7, W8, b8)
        # annotation_pred1 = tf.argmax(conv8, dimension=3, name="prediction1")

        # now to upscale to actual image size
        deconv_shape1 = image_net["pool4"].get_shape()
        W_t1 = utils.weight_variable([4, 4, deconv_shape1[3].value, NUM_OF_CLASSESS], name="W_t1")
        b_t1 = utils.bias_variable([deconv_shape1[3].value], name="b_t1")
        conv_t1 = utils.conv2d_transpose_strided(conv8, W_t1, b_t1, output_shape=tf.shape(image_net["pool4"]))
        fuse_1 = tf.add(conv_t1, image_net["pool4"], name="fuse_1")

        deconv_shape2 = image_net["pool3"].get_shape()
        W_t2 = utils.weight_variable([4, 4, deconv_shape2[3].value, deconv_shape1[3].value], name="W_t2")
        b_t2 = utils.bias_variable([deconv_shape2[3].value], name="b_t2")
        conv_t2 = utils.conv2d_transpose_strided(fuse_1, W_t2, b_t2, output_shape=tf.shape(image_net["pool3"]))
        fuse_2 = tf.add(conv_t2, image_net["pool3"], name="fuse_2")

        shape = tf.shape(image)
        deconv_shape3 = tf.stack([shape[0], shape[1], shape[2], NUM_OF_CLASSESS])
        W_t3 = utils.weight_variable([16, 16, NUM_OF_CLASSESS, deconv_shape2[3].value], name="W_t3")
        b_t3 = utils.bias_variable([NUM_OF_CLASSESS], name="b_t3")
        conv_t3 = utils.conv2d_transpose_strided(fuse_2, W_t3, b_t3, output_shape=deconv_shape3, stride=8)

        annotation_pred = tf.argmax(conv_t3, dimension=3, name="prediction")

    return tf.expand_dims(annotation_pred, dim=3), conv_t3

In [8]:
def train(loss_val, var_list):
    optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate)
    grads = optimizer.compute_gradients(loss_val, var_list=var_list)
    if FLAGS.debug:
        # print(len(var_list))
        for grad, var in grads:
            utils.add_gradient_summary(grad, var)
    return optimizer.apply_gradients(grads)

In [9]:
def main(argv=None):
    keep_probability = tf.placeholder(tf.float32, name="keep_probabilty")
    image = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 3], name="input_image")
    annotation = tf.placeholder(tf.int32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name="annotation")

    pred_annotation, logits = inference(image, keep_probability)
    print("pred annotation : ", pred_annotation.shape)
    print("logits : ", logits.shape)
    tf.summary.image("input_image", image, max_outputs=2)
    tf.summary.image("ground_truth", tf.cast(annotation, tf.uint8), max_outputs=2)
    tf.summary.image("pred_annotation", tf.cast(pred_annotation, tf.uint8), max_outputs=2)
    loss = tf.reduce_mean((tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.squeeze(annotation, squeeze_dims=[3]), name="entropy")))
    #loss = tf.reduce_mean((tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=annotation, name="entropy")))
    tf.summary.scalar("entropy", loss)

    trainable_var = tf.trainable_variables()
    if FLAGS.debug:
        for var in trainable_var:
            utils.add_to_regularization_and_summary(var)
    train_op = train(loss, trainable_var)

    print("Setting up summary op...")
    summary_op = tf.summary.merge_all()

    print("Setting up image reader...")
    #######train_records, valid_records = scene_parsing.read_dataset(FLAGS.data_dir)
    ##print(len(train_records))
    ##print(len(valid_records))
    #######test_records = scene_parsing.read_test_dataset(FLAGS.test_dir)
    ##print(len(test_records))
    train_records = [{'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_T1c.36577/VSD.Brain.XX.O.MR_T1c.36577.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain_3more.XX.O.OT.42811/VSD.Brain_3more.XX.O.OT.42811.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_T2.36578/VSD.Brain.XX.O.MR_T2.36578.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_T1.36576/VSD.Brain.XX.O.MR_T1.36576.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_Flair.36575/VSD.Brain.XX.O.MR_Flair.36575.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat296_0001/VSD.Brain.XX.O.MR_T1c.35946/VSD.Brain.XX.O.MR_T1c.35946.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat296_0001/VSD.Brain_3more.XX.O.OT.42527/VSD.Brain_3more.XX.O.OT.42527.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat296_0001/VSD.Brain.XX.O.MR_T2.35945/VSD.Brain.XX.O.MR_T2.35945.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat296_0001/VSD.Brain.XX.O.MR_T1.35947/VSD.Brain.XX.O.MR_T1.35947.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat296_0001/VSD.Brain.XX.O.MR_Flair.35944/VSD.Brain.XX.O.MR_Flair.35944.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat171_1126/VSD.Brain.XX.O.MR_T1c.35633/VSD.Brain.XX.O.MR_T1c.35633.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat171_1126/VSD.Brain_3more.XX.O.OT.42351/VSD.Brain_3more.XX.O.OT.42351.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat171_1126/VSD.Brain.XX.O.MR_T2.35632/VSD.Brain.XX.O.MR_T2.35632.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat171_1126/VSD.Brain.XX.O.MR_T1.35634/VSD.Brain.XX.O.MR_T1.35634.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat171_1126/VSD.Brain.XX.O.MR_Flair.35631/VSD.Brain.XX.O.MR_Flair.35631.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat135_0001/VSD.Brain.XX.O.MR_T1c.35575/VSD.Brain.XX.O.MR_T1c.35575.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat135_0001/VSD.Brain_3more.XX.O.OT.42307/VSD.Brain_3more.XX.O.OT.42307.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat135_0001/VSD.Brain.XX.O.MR_T2.35574/VSD.Brain.XX.O.MR_T2.35574.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat135_0001/VSD.Brain.XX.O.MR_T1.35576/VSD.Brain.XX.O.MR_T1.35576.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat135_0001/VSD.Brain.XX.O.MR_Flair.35573/VSD.Brain.XX.O.MR_Flair.35573.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat419_0001/VSD.Brain.XX.O.MR_T1c.36290/VSD.Brain.XX.O.MR_T1c.36290.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat419_0001/VSD.Brain_3more.XX.O.OT.42979/VSD.Brain_3more.XX.O.OT.42979.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat419_0001/VSD.Brain.XX.O.MR_T2.36289/VSD.Brain.XX.O.MR_T2.36289.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat419_0001/VSD.Brain.XX.O.MR_T1.36291/VSD.Brain.XX.O.MR_T1.36291.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat419_0001/VSD.Brain.XX.O.MR_Flair.36288/VSD.Brain.XX.O.MR_Flair.36288.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat374_0909/VSD.Brain.XX.O.MR_T1c.36136/VSD.Brain.XX.O.MR_T1c.36136.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat374_0909/VSD.Brain_3more.XX.O.OT.42967/VSD.Brain_3more.XX.O.OT.42967.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat374_0909/VSD.Brain.XX.O.MR_T2.36135/VSD.Brain.XX.O.MR_T2.36135.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat374_0909/VSD.Brain.XX.O.MR_T1.36137/VSD.Brain.XX.O.MR_T1.36137.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat374_0909/VSD.Brain.XX.O.MR_Flair.36134/VSD.Brain.XX.O.MR_Flair.36134.mha'}]
    valid_records = [{'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_T1c.36524/VSD.Brain.XX.O.MR_T1c.36524.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain_3more.XX.O.OT.42775/VSD.Brain_3more.XX.O.OT.42775.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_T2.36525/VSD.Brain.XX.O.MR_T2.36525.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_T1.36523/VSD.Brain.XX.O.MR_T1.36523.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_Flair.36522/VSD.Brain.XX.O.MR_Flair.36522.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat309_0462/VSD.Brain.XX.O.MR_T1c.40985/VSD.Brain.XX.O.MR_T1c.40985.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat309_0462/VSD.Brain_3more.XX.O.OT.42545/VSD.Brain_3more.XX.O.OT.42545.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat309_0462/VSD.Brain.XX.O.MR_T2.40984/VSD.Brain.XX.O.MR_T2.40984.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat309_0462/VSD.Brain.XX.O.MR_T1.40986/VSD.Brain.XX.O.MR_T1.40986.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat309_0462/VSD.Brain.XX.O.MR_Flair.40983/VSD.Brain.XX.O.MR_Flair.40983.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat211_0001/VSD.Brain.XX.O.MR_T1c.35738/VSD.Brain.XX.O.MR_T1c.35738.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat211_0001/VSD.Brain_3more.XX.O.OT.42895/VSD.Brain_3more.XX.O.OT.42895.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat211_0001/VSD.Brain.XX.O.MR_T2.35737/VSD.Brain.XX.O.MR_T2.35737.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat211_0001/VSD.Brain.XX.O.MR_T1.35739/VSD.Brain.XX.O.MR_T1.35739.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat211_0001/VSD.Brain.XX.O.MR_Flair.35736/VSD.Brain.XX.O.MR_Flair.35736.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat314_0001/VSD.Brain.XX.O.MR_T1c.41020/VSD.Brain.XX.O.MR_T1c.41020.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat314_0001/VSD.Brain_3more.XX.O.OT.42569/VSD.Brain_3more.XX.O.OT.42569.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat314_0001/VSD.Brain.XX.O.MR_T2.41019/VSD.Brain.XX.O.MR_T2.41019.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat314_0001/VSD.Brain.XX.O.MR_T1.41021/VSD.Brain.XX.O.MR_T1.41021.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat314_0001/VSD.Brain.XX.O.MR_Flair.41018/VSD.Brain.XX.O.MR_Flair.41018.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat447_0199/VSD.Brain.XX.O.MR_T1c.41137/VSD.Brain.XX.O.MR_T1c.41137.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat447_0199/VSD.Brain_3more.XX.O.OT.42785/VSD.Brain_3more.XX.O.OT.42785.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat447_0199/VSD.Brain.XX.O.MR_T2.41136/VSD.Brain.XX.O.MR_T2.41136.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat447_0199/VSD.Brain.XX.O.MR_T1.41138/VSD.Brain.XX.O.MR_T1.41138.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat447_0199/VSD.Brain.XX.O.MR_Flair.41135/VSD.Brain.XX.O.MR_Flair.41135.mha'}]
    test_records = [{'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_T1c.54271/VSD.Brain.XX.O.MR_T1c.54271.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_T2.54272/VSD.Brain.XX.O.MR_T2.54272.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_T1.54270/VSD.Brain.XX.O.MR_T1.54270.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_Flair.54269/VSD.Brain.XX.O.MR_Flair.54269.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0127/VSD.Brain.XX.O.MR_T1c.40705/VSD.Brain.XX.O.MR_T1c.40705.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0127/VSD.Brain.XX.O.MR_T2.40704/VSD.Brain.XX.O.MR_T2.40704.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0127/VSD.Brain.XX.O.MR_T1.40706/VSD.Brain.XX.O.MR_T1.40706.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0127/VSD.Brain.XX.O.MR_Flair.40703/VSD.Brain.XX.O.MR_Flair.40703.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat123_0360/VSD.Brain.XX.O.MR_T1c.40479/VSD.Brain.XX.O.MR_T1c.40479.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat123_0360/VSD.Brain.XX.O.MR_T2.40478/VSD.Brain.XX.O.MR_T2.40478.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat123_0360/VSD.Brain.XX.O.MR_T1.40480/VSD.Brain.XX.O.MR_T1.40480.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat123_0360/VSD.Brain.XX.O.MR_Flair.40477/VSD.Brain.XX.O.MR_Flair.40477.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0386/VSD.Brain.XX.O.MR_T1c.40757/VSD.Brain.XX.O.MR_T1c.40757.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0386/VSD.Brain.XX.O.MR_T2.40756/VSD.Brain.XX.O.MR_T2.40756.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0386/VSD.Brain.XX.O.MR_T1.40758/VSD.Brain.XX.O.MR_T1.40758.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0386/VSD.Brain.XX.O.MR_Flair.40755/VSD.Brain.XX.O.MR_Flair.40755.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat500_0465/VSD.Brain.XX.O.MR_T1c.41216/VSD.Brain.XX.O.MR_T1c.41216.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat500_0465/VSD.Brain.XX.O.MR_T2.41215/VSD.Brain.XX.O.MR_T2.41215.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat500_0465/VSD.Brain.XX.O.MR_T1.41217/VSD.Brain.XX.O.MR_T1.41217.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat500_0465/VSD.Brain.XX.O.MR_Flair.41214/VSD.Brain.XX.O.MR_Flair.41214.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat114_0115/VSD.Brain.XX.O.MR_T1c.40459/VSD.Brain.XX.O.MR_T1c.40459.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat114_0115/VSD.Brain.XX.O.MR_T2.40458/VSD.Brain.XX.O.MR_T2.40458.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat114_0115/VSD.Brain.XX.O.MR_T1.40460/VSD.Brain.XX.O.MR_T1.40460.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat114_0115/VSD.Brain.XX.O.MR_Flair.40457/VSD.Brain.XX.O.MR_Flair.40457.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0043/VSD.Brain.XX.O.MR_T1c.40701/VSD.Brain.XX.O.MR_T1c.40701.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0043/VSD.Brain.XX.O.MR_T2.40700/VSD.Brain.XX.O.MR_T2.40700.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0043/VSD.Brain.XX.O.MR_T1.40702/VSD.Brain.XX.O.MR_T1.40702.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat456_0043/VSD.Brain.XX.O.MR_Flair.40699/VSD.Brain.XX.O.MR_Flair.40699.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat127_0001/VSD.Brain.XX.O.MR_T1c.40519/VSD.Brain.XX.O.MR_T1c.40519.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat127_0001/VSD.Brain.XX.O.MR_T2.40518/VSD.Brain.XX.O.MR_T2.40518.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat127_0001/VSD.Brain.XX.O.MR_T1.40520/VSD.Brain.XX.O.MR_T1.40520.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat127_0001/VSD.Brain.XX.O.MR_Flair.40517/VSD.Brain.XX.O.MR_Flair.40517.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat464_0001/VSD.Brain.XX.O.MR_T1c.40741/VSD.Brain.XX.O.MR_T1c.40741.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat464_0001/VSD.Brain.XX.O.MR_T2.40740/VSD.Brain.XX.O.MR_T2.40740.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat464_0001/VSD.Brain.XX.O.MR_T1.40742/VSD.Brain.XX.O.MR_T1.40742.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat464_0001/VSD.Brain.XX.O.MR_Flair.40739/VSD.Brain.XX.O.MR_Flair.40739.mha'}, {'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0214/VSD.Brain.XX.O.MR_T1c.40781/VSD.Brain.XX.O.MR_T1c.40781.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0214/VSD.Brain.XX.O.MR_T2.40780/VSD.Brain.XX.O.MR_T2.40780.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0214/VSD.Brain.XX.O.MR_T1.40782/VSD.Brain.XX.O.MR_T1.40782.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_tcia_pat484_0214/VSD.Brain.XX.O.MR_Flair.40779/VSD.Brain.XX.O.MR_Flair.40779.mha'}] 
    #train_records = [{'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_T1c.36577/VSD.Brain.XX.O.MR_T1c.36577.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain_3more.XX.O.OT.42811/VSD.Brain_3more.XX.O.OT.42811.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_T2.36578/VSD.Brain.XX.O.MR_T2.36578.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_T1.36576/VSD.Brain.XX.O.MR_T1.36576.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat473_0001/VSD.Brain.XX.O.MR_Flair.36575/VSD.Brain.XX.O.MR_Flair.36575.mha'}]
    #valid_records = [{'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_T1c.36524/VSD.Brain.XX.O.MR_T1c.36524.mha', 'annotation': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain_3more.XX.O.OT.42775/VSD.Brain_3more.XX.O.OT.42775.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_T2.36525/VSD.Brain.XX.O.MR_T2.36525.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_T1.36523/VSD.Brain.XX.O.MR_T1.36523.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat444_0104/VSD.Brain.XX.O.MR_Flair.36522/VSD.Brain.XX.O.MR_Flair.36522.mha'}]
    #test_records = [{'T1c': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_T1c.54271/VSD.Brain.XX.O.MR_T1c.54271.mha', 'T2': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_T2.54272/VSD.Brain.XX.O.MR_T2.54272.mha', 'T1': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_T1.54270/VSD.Brain.XX.O.MR_T1.54270.mha', 'FLAIR': '/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Testing/HGG_LGG/brats_2013_pat0136_1/VSD.Brain.XX.O.MR_Flair.54269/VSD.Brain.XX.O.MR_Flair.54269.mha'}]


    print("Setting up dataset reader")
    image_options = {'resize': True, 'resize_size': IMAGE_SIZE}
    if FLAGS.mode == 'train':
        train_dataset_reader = dataset.BatchDatset(train_records, image_options)
    validation_dataset_reader = dataset.BatchDatset(valid_records, image_options)
    test_dataset_reader = dataset.TestDatset(test_records, image_options)
      
    sess = tf.Session()

    print("Setting up Saver...")
    saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(FLAGS.logs_dir, sess.graph)
    
    print("Initializing variables...")
    sess.run(tf.global_variables_initializer())
    
    ckpt = tf.train.get_checkpoint_state(FLAGS.logs_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Model restored...")

    if FLAGS.mode == "train":
        for itr in xrange(MAX_ITERATION):
            train_images, train_annotations = train_dataset_reader.next_batch(FLAGS.batch_size)
            feed_dict = {image: train_images, annotation: train_annotations, keep_probability: 0.85}

            sess.run(train_op, feed_dict=feed_dict)

            if itr % 10 == 0:
                train_loss, summary_str = sess.run([loss, summary_op], feed_dict=feed_dict)
                print("Step: %d, Train_loss:%g" % (itr, train_loss))
                summary_writer.add_summary(summary_str, itr)

            if itr % 500 == 0:
                valid_images, valid_annotations = validation_dataset_reader.next_batch(FLAGS.batch_size)
                valid_loss = sess.run(loss, feed_dict={image: valid_images, annotation: valid_annotations,
                                                       keep_probability: 1.0})
                print("%s ---> Validation_loss: %g" % (datetime.datetime.now(), valid_loss))
                saver.save(sess, FLAGS.logs_dir + "model.ckpt", itr)

    elif FLAGS.mode == "valid_visualize":
        print('validation dataset visualization')
        valid_images, valid_annotations = validation_dataset_reader.get_random_batch(FLAGS.batch_size)
        pred = sess.run(pred_annotation, feed_dict={image: valid_images, annotation: valid_annotations,
                                                    keep_probability: 1.0})
        ###valid_annotations = np.squeeze(valid_annotations, axis=3)
        pred = np.squeeze(pred, axis=3)

        for itr in range(FLAGS.batch_size):
            utils.save_image(valid_images[itr].astype(np.double), FLAGS.logs_dir, name="inp_" + str(5+itr))
            utils.save_image(valid_annotations[itr].astype(np.double), FLAGS.logs_dir, name="gt_" + str(5+itr))
            utils.save_image(pred[itr].astype(np.double), FLAGS.logs_dir, name="pred_" + str(5+itr))
            print("Saved image: %d" % itr)

    elif FLAGS.mode == "test_visualize":
        print('test dataset visualization')
        test_images, test_annotations = test_dataset_reader.get_random_batch(FLAGS.batch_size)
        pred = sess.run(pred_annotation, feed_dict={image: test_images, annotation: test_annotations,
                                                    keep_probability: 1.0})
        ###test_annotations = np.squeeze(test_annotations, axis=3)
        ###pred = np.squeeze(pred, axis=3)

        for itr in range(FLAGS.batch_size):
            utils.save_image(test_images[itr].astype(np.double), FLAGS.logs_dir, name="inp_" + str(5+itr))
            utils.save_image(test_annotations[itr].astype(np.double), FLAGS.logs_dir, name="gt_" + str(5+itr))
            utils.save_image(pred[itr].astype(np.double), FLAGS.logs_dir, name="pred_" + str(5+itr))
            print("Saved image: %d" % itr)

In [10]:
tf.app.run()

setting up vgg initialized conv layers ...
Instructions for updating:
Use the `axis` argument instead
pred annotation :  (?, ?, ?, 1)
logits :  (?, ?, ?, 5)
Setting up summary op...
Setting up image reader...
Setting up dataset reader
Initializing Batch Dataset Reader...
(930, 224, 224, 3)
(930, 224, 224, 3)
Initializing Batch Dataset Reader...
(775, 224, 224, 3)
(775, 224, 224, 3)
Initializing Batch Dataset Reader...
(1550, 224, 224, 3)
Setting up Saver...
Initializing variables...


ResourceExhaustedError: OOM when allocating tensor with shape[7,7,512,4096]
	 [[Node: inference/W6/Adam_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@inference/W6"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](inference/W6/Adam_1, inference/W6/Adam/Initializer/zeros)]]

Caused by op 'inference/W6/Adam_1/Assign', defined at:
  File "/home/seonwhee/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/seonwhee/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/seonwhee/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/seonwhee/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/seonwhee/anaconda3/envs/tensorflow/lib/python3.